<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 2 - 1.5 Hours </h1>
<h1 style="text-align:center"> Convolutional Neural Network (CNN) for Handwritten Digits Recognition</h1>

<b> Group name:</b> Sara Giammusso, Samuel Pierre
 
 
The aim of this session is to practice with Convolutional Neural Networks. Each group should fill and run appropriate notebook cells. 


Generate your final report (export as HTML) and upload it on the submission website http://bigfoot-m1.eurecom.fr/teachingsub/login (using your deeplearnXX/password). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed and submitted by May 30th 2018 (23:59:59 CET).

# Introduction

In the previous Lab Session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST data-set. The best achieved accuracy on testing data was about 97%. Can you do better than these results using a deep CNN ?
In this Lab Session, you will build, train and optimize in TensorFlow one of the early Convolutional Neural Networks,  **LeNet-5**, to go to more than 99% of accuracy. 






# Load MNIST Data in TensorFlow
Run the cell below to load the MNIST data that comes with TensorFlow. You will use this data in **Section 1** and **Section 2**.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels
print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

epsilon = 1e-10 # this is a parameter you will use later

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Image Shape: (784,)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


# Section 1 : My First Model in TensorFlow

Before starting with CNN, let's train and test in TensorFlow the example
**y=softmax(Wx+b)** seen in the first lab. 

This model reaches an accuracy of about 92 %.
You will also learn how to launch the TensorBoard https://www.tensorflow.org/get_started/summaries_and_tensorboard to visualize the computation graph, statistics and learning curves. 

<b> Part 1 </b> : Read carefully the code in the cell below. Run it to perform training. 

In [2]:
#STEP 1

# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128
display_step = 1
logs_path = 'log_files/'  # useful for tensorboard

# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

#STEP 2 

# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.flush()

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch:  01   =====> Loss= 1.288405521
Epoch:  02   =====> Loss= 0.732672014
Epoch:  03   =====> Loss= 0.599999217
Epoch:  04   =====> Loss= 0.536443152
Epoch:  05   =====> Loss= 0.497891496
Epoch:  06   =====> Loss= 0.470993731
Epoch:  07   =====> Loss= 0.451451283
Epoch:  08   =====> Loss= 0.435696947
Epoch:  09   =====> Loss= 0.423559110
Epoch:  10   =====> Loss= 0.413112891
Epoch:  11   =====> Loss= 0.404330307
Epoch:  12   =====> Loss= 0.396559129
Epoch:  13   =====> Loss= 0.390292115
Epoch:  14   =====> Loss= 0.384316326
Epoch:  15   =====> Loss= 0.379412651
Epoch:  16   =====> Loss= 0.374492146
Epoch:  17   =====> Loss= 0.370495363
Epoch:  18   =====> Loss= 0.366586237
Epoch:  19   =====> Loss= 0.362921391
Epoch:  20   =====> Loss= 0.359556422
Epoch:  21   =====> Loss= 0.356397682
Epoch:  22   =====> Loss= 0.353948969
Epoch:  23   =====> Loss= 0.351112976
Epoch:  24   =====> Loss= 0.348895340
Epoch:  25   =====> Loss= 0.346488030
Epoch:  26   =====> Loss= 0.344267702
Epoch:  27  

<b> Part 2  </b>: Using Tensorboard, we can  now visualize the created graph, giving you an overview of your architecture and how all of the major components  are connected. You can also see and analyse the learning curves. 

To launch tensorBoard: 
- Open a Terminal and run the command line **"tensorboard --logdir=lab_2/log_files/"**
- Click on "Tensorboard web interface" in Zoe  


Enjoy It !! 


# Section 2 : The 99% MNIST Challenge !

<b> Part 1 </b> : LeNet5 implementation

You are now more familar with **TensorFlow** and **TensorBoard**. In this section, you are to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

Then, you will make some optimizations to get more than 99% of accuracy.

For more informations, have a look at this list of results: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html


<img src="lenet.png",width="800" height="600" align="center">
<center><span>Figure 1: Lenet-5 </span></center>





The LeNet architecture takes a 28x28xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1 - Convolution (5x5):** The output shape should be 28x28x6. **Activation:** ReLU. **MaxPooling:** The output shape should be 14x14x6.

**Layer 2 - Convolution (5x5):** The output shape should be 10x10x16. **Activation:** ReLU. **MaxPooling:** The output shape should be 5x5x16.

**Flatten:** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use tf.reshape.

**Layer 3 - Fully Connected:** This should have 120 outputs. **Activation:** ReLU.

**Layer 4 - Fully Connected:** This should have 84 outputs. **Activation:** ReLU.

**Layer 5 - Fully Connected:** This should have 10 outputs. **Activation:** softmax.


<b> Question 2.1.1 </b>  Implement the Neural Network architecture described above.
For that, your will use classes and functions from  https://www.tensorflow.org/api_docs/python/tf/nn. 

We give you some helper functions for weigths and bias initilization. Also you can refer to section 1. 


In [3]:
# Helper functions  for weigths and bias initilization 
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, stride, padding_):
    return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding=padding_)

In [4]:
def LeNet5_Model(data, transfer="ReLU", keep_prob=1.):    
    
    transferFuncs = {"sigmoid" : tf.sigmoid, "ReLU": tf.nn.relu}
        
    #first convolutional layer
    W_conv1 = weight_variable([5, 5, 1, 6]) ## [filter_width, filter_height, depth_image_in, depth_image_out]
    b_conv1 = bias_variable([6])
    h_conv1 = transferFuncs[transfer](conv2d(data, W_conv1, 1, 'SAME') + b_conv1)
    pool1 = tf.nn.pool(h_conv1, [2,2], "MAX", 'VALID', strides=[2,2])
    
    #second convolutional layer
    W_conv2 = weight_variable([5, 5, 6, 16])
    b_conv2 = bias_variable([16])
    h_conv2 = transferFuncs[transfer](conv2d(pool1, W_conv2, 1, 'VALID') + b_conv2)
    pool2 = tf.nn.pool(h_conv2, [2,2], "MAX", 'VALID', strides=[2,2])
    
    #first fully connected layer
    s = pool2.get_shape().as_list()
    flattened_length = s[1] * s[2] * s[3]
    pool2_flat = tf.reshape(pool2, [-1, flattened_length])
    W_fc1 = weight_variable([1*5*5*16, 120])
    b_fc1 = bias_variable([120])
    h_fc1 = transferFuncs[transfer](tf.matmul(pool2_flat, W_fc1) + b_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    #second fully connected layer
    W_fc2 = weight_variable([120, 84])
    b_fc2 = bias_variable([84])
    h_fc2 = transferFuncs[transfer](tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)
    
    #third fully connected layer
    W_fc3 = weight_variable([84, 10])
    b_fc3 = bias_variable([10])
    h_fc3 = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)
    
    return h_fc3

<b> Question 2.1.2. </b>  Calculate the number of parameters of this model 

In [5]:
# first conv
pconv1 = 5*5*1*6 # filter_height * filter_width * channels_in * num_feature_maps
# second conv
pconv2 = 5*5*1*16 # filter_height * filter_width * channels_in * num_feature_maps
# first fcl
pfcl1 = 5*5*16*120 # fcl_input_size * fcl_output_size
# second fcl
pfcl2 = 84*120 # fcl_input_size * fcl_output_size
# third fcl
pfcl3 = 84*10 # fcl_input_size * fcl_output_size
pbias = 6+16+120+84+10 # all the biases
total = pbias + pfcl1 + pfcl2 + pfcl3 + pconv2 + pconv1
print(total)

59706


<b> Question 2.1.3. </b>  Define your model, its accuracy and the loss function according to the following parameters (you can look at Section 1 to see what is expected):

     Learning rate: 0.001
     Loss Fucntion: Cross-entropy
     Optimizer: tf.train.GradientDescentOptimizer
     Number of epochs: 40
     Batch size: 128

In [6]:
tf.reset_default_graph() # reset the default graph before defining a new model

# Parameters
learning_rate = 0.001
training_epochs = 40
batch_size = 128

display_step = 10
logs_path = 'log_files/'

<b> Question 2.1.4. </b>  Implement the evaluation function for accuracy computation 

In [7]:
def evaluate(model, y):
    correct = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    return tf.reduce_mean(tf.cast(correct, tf.float32))

<b> Question 2.1.5. </b>  Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 100 epochs)
- Save the model after training
- Print after training the final testing accuracy 



In [27]:
import time 
def train(learning_rate, training_epochs, batch_size, display_step = 1, \
          logs_path='log_files/', optFunction="SGD", verbose=True, transfer="ReLU", keep_probability= 1.0):
    
    optFunctions = {"SGD":tf.train.GradientDescentOptimizer, "Adam":tf.train.AdamOptimizer}
    
    # Erase previous graph
    tf.reset_default_graph()

    x = tf.placeholder(tf.float32, [None, 28, 28, 1], name='InputData')
    y = tf.placeholder(tf.float32, [None, 10], name='LabelData')
    keep_prob = tf.placeholder(tf.float32)

    # Construct model
    with tf.name_scope('Model'):
        pred = LeNet5_Model(x, transfer=transfer)

    # Define loss and optimizer
    with tf.name_scope('Loss'):
        cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
        #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))

    with tf.name_scope(optFunction):
        if transfer is "sigmoid":
            optimizer = optFunctions[optFunction](learning_rate).minimize(cost)
        else:
            opt = optFunctions[optFunction](learning_rate)
            gvs = opt.compute_gradients(cost)
            capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
            optimizer = opt.apply_gradients(capped_gvs)

    # Evaluate model
    with tf.name_scope('Accuracy'):
        accuracy = evaluate(pred, y)

    # Initializing the variables
    init = tf.global_variables_initializer()

    # Create a summary to monitor cost tensor
    tf.summary.scalar("Loss", cost)
    # Create a summary to monitor accuracy tensor
    tf.summary.scalar("Accuracy", accuracy)
    # Merge all summaries into a single op
    merged_summary_op = tf.summary.merge_all()

    x_val, y_val = mnist.validation.images.reshape(-1, 28, 28, 1), mnist.validation.labels
    x_test, y_test = mnist.test.images.reshape(-1, 28, 28, 1), mnist.test.labels

    with tf.Session() as sess:
        #acc_history = []
        test_history = []
        val_history = []
        #train_history = []

        sess.run(init)
        if verbose is True:
            print("Start Training!")
        # op to write logs to Tensorboard
        summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        saver = tf.train.Saver()
        # Training cycle
        start_time = time.time()
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)
            # Loop over all batches
            for i in range(total_batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                batch_xs = batch_xs.reshape(-1, 28, 28, 1)
                # Run optimization op (backprop), cost op (to get loss value)
                # and summary nodes
                _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                         feed_dict={x: batch_xs, y: batch_ys, keep_prob: keep_probability})
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch * total_batch + i)
                # Compute average loss
                avg_cost += c / total_batch
            # Display logs per epoch step
            #train_acc = accuracy.eval({x: batch_xs, y:batch_ys})
            val_acc = accuracy.eval({x: x_val, y:y_val, keep_prob:keep_probability})
            test_acc = accuracy.eval({x: x_test, y:y_test, keep_prob:keep_probability})

            #train_history.append(train_acc)
            val_history.append(val_acc)
            test_history.append(test_acc)
            
            saver.save(sess, 'Models/model_' + str(learning_rate) + '_' + str(batch_size) + '_' + optFunction)
            if verbose is True and (epoch+1) % display_step == 0:
                print("Epoch: ", '%02d' % (epoch+1), \
                      "  =====> Loss=", "{:.9f}".format(avg_cost), \
                      " Validation accuracy=", val_acc, " Test accuracy=", test_acc)
            if test_acc >= 0.99:
                if verbose is True:
                    print("Test Accuracy over 99%% reached after %d epochs" %(epoch+1))
                break
        elapsed_time = time.time() - start_time       
        if verbose is True:
            print("Training Finished!")
            # Test model
            # Calculate accuracy
            print("Test accuracy:", accuracy.eval({x: x_test, y:y_test, keep_prob: keep_probability}))
            print("Elapsed time: ", elapsed_time, "sec")
        
    return val_history, test_history, elapsed_time

In [23]:
val_hist1, test_hist1, elapsed_time_1 = train(learning_rate, training_epochs, batch_size)

Start Training!
Epoch:  01   =====> Loss= 2.277477276  Validation accuracy= 0.1878  Test accuracy= 0.1866
Epoch:  02   =====> Loss= 2.228345729  Validation accuracy= 0.3138  Test accuracy= 0.3174
Epoch:  03   =====> Loss= 2.145544900  Validation accuracy= 0.4588  Test accuracy= 0.4535
Epoch:  04   =====> Loss= 1.961876166  Validation accuracy= 0.644  Test accuracy= 0.6419
Epoch:  05   =====> Loss= 1.552841382  Validation accuracy= 0.7506  Test accuracy= 0.7512
Epoch:  06   =====> Loss= 1.005656833  Validation accuracy= 0.8094  Test accuracy= 0.8136
Epoch:  07   =====> Loss= 0.682512069  Validation accuracy= 0.8446  Test accuracy= 0.852
Epoch:  08   =====> Loss= 0.545619721  Validation accuracy= 0.8674  Test accuracy= 0.8725
Epoch:  09   =====> Loss= 0.471422312  Validation accuracy= 0.8794  Test accuracy= 0.887
Epoch:  10   =====> Loss= 0.423888842  Validation accuracy= 0.8908  Test accuracy= 0.8967
Epoch:  11   =====> Loss= 0.382332977  Validation accuracy= 0.9  Test accuracy= 0.904
E

<b> Question 2.1.6 </b> : Use TensorBoard to visualise and save loss and accuracy curves. 
You will save figures in the folder **"lab_2/MNIST_figures"** and display them in your notebook.

Please put your loss and accuracy curves here.

<b> Part 2 </b> : LeNET 5 Optimization


<b> Question 2.2.1 </b>

- Retrain your network with AdamOptimizer and then fill the table above:


| Optimizer            |  Gradient Descent  |    AdamOptimizer    |
|----------------------|--------------------|---------------------|
| Testing Accuracy     |         ???        |        ???          |       
| Training Time        |         ???        |        ???          |  

- Which optimizer gives the best accuracy on test data?

**Your answer:** ...


In [28]:
tf.reset_default_graph()
val_hist_2, test_hist_2, elapsed_time_2 = train(learning_rate, training_epochs, batch_size, optFunction="Adam")

Start Training!
Epoch:  01   =====> Loss= 0.393101458  Validation accuracy= 0.9724  Test accuracy= 0.9738
Epoch:  02   =====> Loss= 0.084432033  Validation accuracy= 0.9776  Test accuracy= 0.9778
Epoch:  03   =====> Loss= 0.061928234  Validation accuracy= 0.984  Test accuracy= 0.9833
Epoch:  04   =====> Loss= 0.047621400  Validation accuracy= 0.9866  Test accuracy= 0.9859
Epoch:  05   =====> Loss= 0.041457841  Validation accuracy= 0.9886  Test accuracy= 0.9868
Epoch:  06   =====> Loss= 0.033015398  Validation accuracy= 0.9856  Test accuracy= 0.9865
Epoch:  07   =====> Loss= 0.029921563  Validation accuracy= 0.9886  Test accuracy= 0.9887
Epoch:  08   =====> Loss= 0.026494493  Validation accuracy= 0.9862  Test accuracy= 0.989
Epoch:  09   =====> Loss= 0.021423107  Validation accuracy= 0.9918  Test accuracy= 0.99
Test Accuracy over 99% reached after 9 epochs
Training Finished!
Test accuracy: 0.99
Elapsed time:  157.9079737663269 sec


<b> Question 2.2.2</b> Try to add dropout (keep_prob = 0.75) before the first fully connected layer. You will use tf.nn.dropout for that purpose. What accuracy do you achieve on testing data?

**Accuracy achieved on testing data:** ...

In [29]:
tf.reset_default_graph()
val_hist_3, test_hist_3, elapsed_time_3 = train(learning_rate, training_epochs, batch_size, optFunction="Adam", keep_probability= 0.75)

Start Training!
Epoch:  01   =====> Loss= 0.356263814  Validation accuracy= 0.9692  Test accuracy= 0.9674
Epoch:  02   =====> Loss= 0.090154480  Validation accuracy= 0.98  Test accuracy= 0.9811
Epoch:  03   =====> Loss= 0.061317494  Validation accuracy= 0.9842  Test accuracy= 0.9838
Epoch:  04   =====> Loss= 0.046231128  Validation accuracy= 0.987  Test accuracy= 0.9871
Epoch:  05   =====> Loss= 0.036664538  Validation accuracy= 0.9882  Test accuracy= 0.9885
Epoch:  06   =====> Loss= 0.033082531  Validation accuracy= 0.9852  Test accuracy= 0.9861
Epoch:  07   =====> Loss= 0.027664797  Validation accuracy= 0.986  Test accuracy= 0.9882
Epoch:  08   =====> Loss= 0.023629387  Validation accuracy= 0.9892  Test accuracy= 0.9904
Test Accuracy over 99% reached after 8 epochs
Training Finished!
Test accuracy: 0.9904
Elapsed time:  140.13831901550293 sec
